## RNN入門

目的  
RNNのアルゴリズムを大体理解し、できればデータセットを用いて実際モデリングしたい  

### RNNとは  
まず、RNNは何ぞや？  
再帰型ニューラルネットと呼ばれるやつです。  
https://deepage.net/deep_learning/2017/05/23/recurrent-neural-networks.html  
https://qiita.com/kiminaka/items/87afd4a433dc655d8cfd  

大まかにいうと、CNNを用いて、写真が犬か猫かを識別します。  
180×180ピクセルの画像を入力して、モデリングしたら、犬か猫かのベクトルが出てきます。  
でも、現実世界は、犬猫だけではない、180×180でもない、  
可変長のインプットとアウトプットを取り扱うのがこのRNNです。  
イメージとしては下記のようです。  

<img src='./RNN/1.jpg' width = 300>

新しいデータx1 x2 x3 xnがあったら、新しいデータを入れて、モデルを微調整❓する形です。

自然言語処理、機械翻訳、音声認識、画像の概要生成(すごい)などがあります。  

### simpleRNN
<img src='./RNN/rnn-top.jpg' width = 300 >
一番シンプルなRNNは上記のようなイメージ  
時点tで、入力値Xtと前回時点の状態St-1から新しいStに遷移  
この時ytが生成されます。   

勾配消失の問題？？に直面している  
### LSTM  
<img src='./RNN/lstm-rnn.jpg' width = 300 >

simpleRNNの問題は、長期的なデータは勉強できない、  
LSTMは解消できた  
「ゲート」と呼ばれる情報の取捨選択機構がある  
sigmoid関数で、ゲートを閉まるかどうかをコントロール  

__勾配消失__  http://nnadl-ja.github.io/nnadl_site_ja/chap5.html

### 実装例  
では、何度ものminist  

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

C:\Users\ko_kaku\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


この画像を上部から下部に向けて遷移させながらLSTMで学習します。
<img src='./RNN/mnist-gif.gif' width = 300 >
1行ずつスキャンと想像します 

In [2]:
mnist.train.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [3]:
import matplotlib.pyplot as plt 
%matplotlib inline


In [4]:
x = tf.placeholder("float", [None, 28, 28])
y = tf.placeholder("float", [None, 10])

In [5]:
x = tf.unstack(x, 28, 1)

# LSTMセルを定義する
lstm_cell = rnn.BasicLSTMCell(128, forget_bias=1.0)

# モデルの定義。各タイムステップの出力値と状態が返される
outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # 変数定義
weight = tf.Variable(tf.random_normal([128, 10]))
bias = tf.Variable(tf.random_normal([10]))


preds = tf.matmul(outputs[-1], weight) + bias


In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# 評価用
correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
batch_size = 128
n_training_iters = 100000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < n_training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # next_batchで返されるbatch_xは[batch_size, 784]のテンソルなので、batch_size×28×28に変換します。
        batch_x = batch_x.reshape((batch_size, 28, 28))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % 10 == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print('step: {} / loss: {:.6f} / acc: {:.5f}'.format(step, loss, acc))
        step += 1

    # テスト
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, 28, 28))
    test_label = mnist.test.labels[:test_len]
    test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
    print("Test Accuracy: {}".format(test_acc))

TypeError: unhashable type: 'list'

### コードわからない！

In [15]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file).read()
text[0:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [ ]:
https://colah.github.io/posts/2015-08-Understanding-LSTMs/